<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/few_shot_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deeppavlov

     |████████████████████████████████| 993kB 6.6MB/s 
     |████████████████████████████████| 7.3MB 19.0MB/s 
     |████████████████████████████████| 512kB 34.9MB/s 
     |████████████████████████████████| 10.4MB 35.3MB/s 
     |████████████████████████████████| 61kB 5.9MB/s 
     |████████████████████████████████| 8.2MB 624kB/s 
     |████████████████████████████████| 655kB 33.2MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 2.1MB 33.2MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
     |████████████████████████████████| 61kB 5.7MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 3.8MB 34.9MB/s 
     |████████████████████████████████| 860kB 32.9MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 


In [ ]:
!pip install tensorflow==1.15.0

In [ ]:
!apt-get --yes install git

In [ ]:
!pip install git+https://github.com/deepmipt/bert.git@feat/multi_gpu

In [ ]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/few_test.txt

In [ ]:
!mv few_test.txt train.txt

In [ ]:
import json
from deeppavlov import configs, build_model, train_model

with configs.ner.ner_ontonotes_bert_mult.open(encoding='utf8') as f:
    ner_config = json.load(f)

ner_config['dataset_reader']['data_path'] = './'  # directory with train.txt, valid.txt and test.txt files
ner_config['metadata']['variables']['NER_PATH'] = './'
# ner_config['metadata']['download'] = [ner_config['metadata']['download'][-1]]  # do not download the pretrained ontonotes model

In [ ]:
ner_model = train_model(ner_config, download=True)

In [ ]:
ner_model(["какое-то не лекарство", "пью леонезит 1 месяц", "варкрафт 2 интересная игра"])